In [ ]:
import sys
!{sys.executable} -m pip install matplotlib
import json
from collections import Counter

with open("kassite_names_db_analyzed.json", "r", encoding="utf-8") as f:
    data = json.load(f)
def finde_elemente(element): 
    ergebnisse = [e['transcription'] for e in data if element in e['tokens']]
    return ergebnisse

alle_tokens = [t for e in data for t in e['tokens']]
token_stats = Counter(alle_tokens)

print(f"--- Statisitk für {len(data)} Namen ---")
print(f"Häufigste Laute: {token_stats.most_common(5)}")

print(f"\nNamen mit 'b' und 'u' Folge:")
for name in finde_elemente('b'):
    if 'u' in name:
        print(f" -> {name}")

In [ ]:
from collections import Counter


bigrams = []

for eintrag in data:
    t = eintrag.get('tokens', [])
    

    if len(t) >= 2:
        paare = [t[i] + t[i+1] for i in range(len(t)-1)]
        bigrams.extend(paare)


bigram_stats = Counter(bigrams)

print("\n🔍 SILBEN-MINING (Top 15 Buchstaben-Paare):")
print("-" * 45)
for paar, count in bigram_stats.most_common(15):
    print(f" Silbe/Paar: '{paar}' | {count} mal gefunden")

strukturen = Counter([e.get('structure', 'NA') for e in data])
print("\n HÄUFIGSTE WORT-STRUKTUREN:")
print("-" * 45)
for struct, count in strukturen.most_common(5):
    print(f" Muster: {struct:<12} | {count} Namen")

In [ ]:
from collections import Counter
anfaenge = []
enden = []

for eintrag in data:
    t = eintrag.get("tokens", [])
    if len(t) >= 2:
        anfaenge.append(t[0] + t[1])
        enden.append(t[-2] + t[-1])

stats_anfang = Counter(anfaenge)
stats_ende = Counter(enden)

print("POSITIONALE ANALYSE (Top 10)")
print("-" * 45)
print(f"{'Silbe':<10} | {'Als Anfang':<12} | {'Als Ende':<12}")

top_silben = [s for s, c in bigram_stats.most_common(15)]

for s in top_silben:
    a_count = stats_anfang[s]
    e_count = stats_ende[s]
    print(f"{s:<10} | {a_count:<12} | {e_count:<12}") 

In [ ]:
import json

file_name= "kassite_names_db_analyzed.json"

try:
    with open(file_name, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f" {len(data)} Namen erfolgreich geladen.")
except FileNotFoundError:
    print(f" Datei '{file_name}' nicht gefunden. Bitte lade sie hoch.")
    
kassite_gods = ["Bugaš", "Šuriaš", "Maruttaš", "Buriaš", "Indaš", "Hala", "Šugab", "Sah", "Harbe"]

def analyze_theophoric_elements(data, gods_list):
    results = []
    
    for entry in data:
        name = entry ['transcription']
        found_gods = []
        
        for god in gods_list:
            if god.lower() in name.lower():
                found_gods.append(god)
        if found_gods:
            is_prefix = any(name.lower().startswith(g.lower()) for g in found_gods)
            
            results.append({
                'name': name,
                'gods': found_gods,
                'position': 'Anfang' if is_prefix else "Ende/Mitte"
            })
    return results
theophoric_names = analyze_theophoric_elements(data, kassite_gods)

print(f" Theophore Analyse")
print(f"Von {len(data)} Namen enthalten {len(theophoric_names)} ein Götter-Element.\n")

print(f"{'Name':<20} | {'Gott':<12} | {'Position'}")
print("-" * 50)

for item in theophoric_names[:15]:
    gods_str = ", ".join(item['gods'])
    print(f"{item['name']:<20} | {gods_str:<12} | {item['position']}")

import matplotlib.pyplot as plt
from collections import Counter


all_found_gods = []
for item in theophoric_names:
    all_found_gods.extend(item['gods']) 


god_counts = Counter(all_found_gods)
common_gods = god_counts.most_common()


names = [x[0] for x in common_gods] 
counts = [x[1] for x in common_gods]


plt.figure(figsize=(10, 6))
plt.bar(names, counts, color='skyblue', edgecolor='navy')
plt.title('Häufigkeit der Gottheiten in den kassitischen Namen', fontsize=14)
plt.xlabel('Gottheit', fontsize=12)
plt.ylabel('Anzahl der Nennungen', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)


for i, v in enumerate(counts):
    plt.text(i, v + 0.2, str(v), ha='center', fontweight='bold') # KORREKTUR: Komma und fontweight

plt.tight_layout()
plt.show()

